In [61]:
from glob import glob
import pandas as pd
import shutil
from tqdm import tqdm
import torch
import torch.nn
from PIL import Image

# TO DO

# -- Download the data 
# -- Extract the file
# -- Split the File

# Pytorch Modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

#General Modules
from PIL import Image
from tqdm import tqdm
from time import time, sleep
import pandas as pd
import numpy as np 
import os
import sys

In [53]:
all_images = glob('Dataset/*/*/*/*.png')

In [54]:
emotions = {'anger': 1, 'disgust':2,'fear':3, 'joy':4, 'neutral':5, 'sadness':6, 'surprise':7}

In [57]:
loop1 = tqdm(enumerate(emotions.keys()))
for i, emotion_key in loop1:
    loop1.set_description(f"[{i+1} / {len(emotions.keys())}] :: {emotion_key}")
    emotion_key_filtered = list(filter(lambda x: emotion_key in x, all_images))
    loop2 = tqdm(enumerate(emotion_key_filtered), leave = False)
    for j , img in loop2:
        loop2.set_description(f"Conversion/Movment[{j+1} / {len(emotion_key_filtered)}]")
        shutil.copy(img,f'Dataset_custom_build/{emotion_key}/{j+1}.png')
#         pass

[1 / 7] :: anger: : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
245it [00:00, 2447.77it/s]
540it [00:00, 2577.68it/s]
832it [00:00, 2671.09it/s]
1119it [00:00, 2726.76it/s]
1409it [00:00, 2767.44it/s]
1707it [00:00, 2827.29it/s]
2012it [00:00, 2885.46it/s]
2328it [00:00, 2960.64it/s]
2646it [00:00, 3022.31it/s]
2963it [00:01, 3061.11it/s]
3264it [00:01, 2987.86it/s]
3560it [00:01, 2944.76it/s]
3864it [00:01, 2970.87it/s]
4185it [00:01, 3037.57it/s]
4496it [00:01, 3056.72it/s]
4802it [00:01, 2937.22it/s]
5097it [00:01, 2861.80it/s]
5385it [00:01, 2681.38it/s]
5679it [00:01, 2753.88it/s]
5971it [00:02, 2801.59it/s]
6274it [00:02, 2865.66it/s]
6563it [00:02, 2800.13it/s]
6864it [00:02, 2858.48it/s]
7174it [00:02, 2926.55it/s]
7480it [00:02, 2964.04it/s]
7778it [00:02, 2932.32it/s]
8073it [00:02, 2851.05it/s]
8363it [00:02, 2862.38it/s]
8656it [00:02, 2881.04it/s]
8951it [00:03, 2893.16it/s]
[2 / 7] :: disgust: : 1it [00:03,  3.18s/it]
0it [00:00, ?it/s]
227it [00:00, 2263.93it/s]
441it [00:00, 

In [108]:
class FERG_2D_256_x_256_Dataset(Dataset):
    def __init__(self):
        self.imgdata = []
        self.labels = []
        self.arr = None 

        for emotion_key in tqdm(emotions.keys(), desc = "Dataset creations", leave = False):
            
            emotion_val = emotions[emotion_key]
            emotion_key_filtered = list(filter(lambda x: emotion_key in x, all_images))
            
            for img in tqdm(emotion_key_filtered[:5], leave = False):
                
                self.arr = Image.open(img).convert('RGB')
                self.arr = self.arr.resize((224, 224))
#                 print(self.arr.dtype)
#                 print(self.arr.shape)
                self.arr = np.asarray(self.arr)
                self.arr = self.arr.reshape((3,224,224))
                self.imgdata.append(self.arr)
                self.labels.append(emotion_val)

        self.imgdata = np.array(self.imgdata)

        self.X =torch.tensor(self.imgdata,  dtype = torch.float32)
        self.Y =torch.tensor(np.array(self.labels),dtype = torch.long)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()  
        return self.X[idx], self.Y[idx] 



In [109]:
data = FERG_2D_256_x_256_Dataset()

  0%|          | 0/5 [00:00<?, ?it/s]
                                     
  0%|          | 0/5 [00:00<?, ?it/s]
                                     
  0%|          | 0/5 [00:00<?, ?it/s]
                                     
  0%|          | 0/5 [00:00<?, ?it/s]
                                     
  0%|          | 0/5 [00:00<?, ?it/s]
                                     
  0%|          | 0/5 [00:00<?, ?it/s]


In [110]:
loader = DataLoader(data    , batch_size = 1,  shuffle = True)

In [111]:
losses = []
accuracies = []
step = 0
device = 'cpu'
for epoch in range(3):                # Initializing or setting to the training method
    loop = tqdm(enumerate(loader), total = len(loader), leave = False)
    for batch_idx, (data, target) in loop:
        data, target = data.to(device), target.to(device)
        print(data.shape)
        print(target)
        break
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        accuracy = accuracy_check(output,target)
        loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
        loop.set_postfix(loss = loss.item(), accu = "{0:.0%}".format(accuracy))            
        writer.add_scalar('Trainig loss', loss.item(), global_step = step)
        writer.add_scalar('Trainig Acc.', accuracy, global_step = step)

        step +=1
        sleep(0.002)

torch.Size([1, 3, 224, 224])
tensor([2])
torch.Size([1, 3, 224, 224])
tensor([3])
torch.Size([1, 3, 224, 224])
tensor([7])
